In [1]:
from etl.spark.spark_session_helper import spark

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor ,GBTRegressionModel
from pyspark.ml import Pipeline, PipelineModel

features = [
     'is_male',
     'batter_runs_30D',
     'batter_runs_90D',
     'batter_runs_300D',
     'batter_runs_1000D',
     'batter_runs_1000D_venue',
     'balls_faced_30D',
     'balls_faced_90D',
     'balls_faced_300D',
     'balls_faced_1000D',
     'balls_faced_1000D_venue',
     'dismissals_30D',
     'dismissals_90D',
     'dismissals_300D',
     'dismissals_1000D',
     'dismissals_1000D_venue',
     'boundary_count_30D',
     'boundary_count_90D',
     'boundary_count_300D',
     'boundary_count_1000D',
     'boundary_count_1000D_venue',
     'six_count_30D',
     'six_count_90D',
     'six_count_300D',
     'six_count_1000D',
     'six_count_1000D_venue',
     'batting_avg_30D',
     'batting_avg_90D',
     'batting_avg_300D',
     'batting_avg_1000D',
     'batting_avg_1000D_venue',
     'batting_sr_30D',
     'batting_sr_90D',
     'batting_sr_300D',
     'batting_sr_1000D',
     'batting_sr_1000D_venue',
     'total_runs_30D',
     'total_runs_90D',
     'total_runs_300D',
     'total_runs_1000D',
     'total_runs_1000D_venue',
     'deliveries_30D',
     'deliveries_90D',
     'deliveries_300D',
     'deliveries_1000D',
     'deliveries_1000D_venue',
     'wicket_sum_30D',
     'wicket_sum_90D',
     'wicket_sum_300D',
     'wicket_sum_1000D',
     'wicket_sum_1000D_venue',
     'maiden_count_30D',
     'maiden_count_90D',
     'maiden_count_300D',
     'maiden_count_1000D',
     'maiden_count_1000D_venue',
     'bowling_avg_30D',
     'bowling_avg_90D',
     'bowling_avg_300D',
     'bowling_avg_1000D',
     'bowling_avg_1000D_venue',
     'bowling_sr_30D',
     'bowling_sr_90D',
     'bowling_sr_300D',
     'bowling_sr_1000D',
     'bowling_sr_1000D_venue',
     'bowling_eco_30D',
     'bowling_eco_90D',
     'bowling_eco_300D',
     'bowling_eco_1000D',
     'bowling_eco_1000D_venue'
]

23/02/25 22:21:04 WARN Utils: Your hostname, Chaturvedi_PC resolves to a loopback address: 127.0.1.1; using 172.31.235.42 instead (on interface eth0)
23/02/25 22:21:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/abhay/.ivy2/cache
The jars for the packages stored in: /home/abhay/.ivy2/jars
org.jpmml#pmml-sparkml added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5992e9ae-e1b9-400f-8053-dbbaf48c9404;1.0
	confs: [default]
	found org.jpmml#pmml-sparkml;2.1.0 in central
	found org.jpmml#pmml-converter;1.5.3 in central
	found org.jpmml#pmml-model-metro;1.6.3 in central
	found org.jpmml#pmml-model;1.6.3 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.13.1 in local-m2-cache
	[2.13.1] com.fasterxml.jackson.core#jackson-annotations;[2.9.4, 2.13.1]
	found jakarta.xml.bind#jakarta.xml.bind-api;3.0.1 in local-m2-cache
	found org.glassfish.jaxb#jaxb-runtime;3.0.2 in local-m2-cache
	found com.sun.activation#jakarta.activation;2.0.1 in local-m2-cache
	found org.glassfish.jaxb#jaxb-core;3.0.2 in local-m2-cache
	found com.sun.istack#istack-commons-runtime;4.0.1 in local-m2-cache
	found com.google.guava#guava;21.0 in local-m2

In [2]:
from path_manager import model_train_input_path, model_test_input_path, model_train_predictions_path, model_test_predictions_path, model_pipeline_save_artifact_path

train_df = spark.read.parquet(model_train_input_path)
test_df = spark.read.parquet(model_test_input_path)
print("Starting Model Training Flow")
# assemble features
vector_assembler = VectorAssembler(inputCols = features, outputCol='features')
# GBT regressor
gbt_regressor = GBTRegressor(featuresCol='features', labelCol='fantasy_points', maxIter=10)
# create Pipeline
pipeline = Pipeline(stages = [vector_assembler, gbt_regressor])
pipeline_model = pipeline.fit(train_df)

print("Pipeline model trained")
# save model
pipeline_model.write().overwrite().save(model_pipeline_save_artifact_path)

print("model saved")

# load saved model
loaded_pipeline_model = PipelineModel.load(model_pipeline_save_artifact_path)

print("loaded saved model")

# assemble test features and predict on test data
test_predictions = loaded_pipeline_model.transform(test_df)
test_predictions.limit(100).toPandas()

# # write predictions to disk
# print("writing predictions over test input data to disk")
# test_predictions.write.format("parquet").partitionBy(["dt"]).mode("overwrite").save(model_test_predictions_path)

# print("writing predictions over train input data to disk")
# train_predictions.write.format("parquet").partitionBy(["dt"]).mode("overwrite").save(model_train_predictions_path)

/home/abhay/work/dream11


Starting Model Training Flow


Pipeline model trained
model saved
loaded saved model


23/02/25 22:21:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/02/25 22:21:38 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/25 22:21:38 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


,batter_name,batter_run_sum,balls_faced,dismissals,boundary_count,six_count,batter_runs_30D,batter_runs_90D,batter_runs_300D,batter_runs_1000D,...,fielding_wicket_sum,fielding_wicket_sum_300D,fielding_wicket_sum_1000D,player_id,dt,match_id,is_male,fantasy_points,features,prediction
0,None,0,0,0,0,0,0,0,0,0,...,1,1,21,ea27a674,2018-07-08,1127467,1,11,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.487413
1,None,0,0,0,0,0,0,0,0,0,...,3,7,21,9c376401,2021-10-21,1273721,1,25,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.487413
2,None,0,0,0,0,0,0,0,0,0,...,1,3,4,d85b0f8b,2019-12-31,1195589,1,11,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.487413
3,None,0,0,0,0,0,0,0,0,0,...,1,4,11,7fa12533,2021-04-01,1233981,1,11,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.487413
4,None,0,0,0,0,0,0,0,0,0,...,1,1,1,a699488a,2023-01-14,1351062,1,11,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.487413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,None,0,0,0,0,0,0,0,0,0,...,0,0,0,d6164dc0,2019-03-29,wi_201706,0,33,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",33.510988
96,None,0,0,0,0,0,0,0,0,0,...,0,0,0,8d5d991e,2022-02-01,1296699,1,54,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",56.434879
97,None,0,0,0,0,0,0,0,0,0,...,0,0,0,94b30fa5,2018-06-23,1126722,0,31,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16.452596
98,None,0,0,0,0,0,0,0,0,0,...,0,0,0,c7a995d3,2019-11-09,1205735,1,124,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.595163


In [4]:
from path_manager import model_pipeline_save_artifact_path, model_pipeline_pmml_save_artifact_path
from pyspark2pmml import PMMLBuilder

pmmlBuilder = PMMLBuilder(spark, train_df, loaded_pipeline_model)

pmmlBuilder.buildFile(model_pipeline_pmml_save_artifact_path)

'/home/abhay/work/dream11/model_data/models/GbtRegressionPipelinePMML/v1/dream11_v1.pmml'